# Sentiment Analysis IMDB Dataset

This program attains 90% accuracy on sentiment classification (positive/negative) of the IMDB dataset.

## Load Data

In [1]:
import nlp_proj_utils as utils

# Function for loading imdb dataset
def load_imdb():
    train, test = utils.get_imdb_dataset()
    TEXT_COL, LABEL_COL = 'text', 'sentiment'
    return (
        train[TEXT_COL], train[LABEL_COL],
        test[TEXT_COL], test[LABEL_COL])

In [2]:
train_text, train_label, test_text, test_label = load_imdb()

data already available, skip downloading.
imdb loaded successfully.


In [3]:
# Check Shape, should not throw exceptions
for data in train_text, train_label, test_text, test_label:
    assert data.shape == (25000,)

## Prepare Data 

### Build Vectorizer

We use the build-in method for generating vocabulary and doing vectorization provided by `TfidfVectorizer` from Sklearn.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(
    min_df=2, # ignore word that only appears in 1 document
    ngram_range=(1, 2), # consider both uni-gram and bi-gram
)

In [6]:
# Learn (fit) and transform text into vector
train_x = tfidf_vectorizer.fit_transform(train_text)

# Convert label to 0 and 1 (optional)
train_y = train_label.apply(lambda x: 1 if x == 'pos' else 0)

In [7]:
# Check the shape
print('Training x shape:', train_x.shape)
print('Training y shape:', train_y.shape)

Training x shape: (25000, 438350)
Training y shape: (25000,)


Notice the very high dimensionality of the training set. We will use word embeddings to resolve this.

In [8]:
# Expect 12500 for 1 and 0, instead of pos and neg
train_y.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [9]:
# Apply the same transformer to validation set as well
# Simply call `transform` this time, don't do `fit` again
test_x = tfidf_vectorizer.transform(test_text)
test_y = test_label.apply(lambda x: 1 if x == 'pos' else 0)

In [10]:
# Sanity check
assert test_x.shape == train_x.shape
assert test_y.shape == train_y.shape

### Dimensionality Reduction

There are a couple of ways we could reduce the dimensionality.

- One is to set a hard cut off on the number words in our vocabulary. For instance, keep the top 3000 frequent words in the vocabulary. 
- The other way is to use a pretrained word embeddings. This is based on transfer learning, where we reduce the dimensions by learning the relation between words from another training task (e.g.: Word2Vec). 

Pros and Cons of the Two Methods: 

- One defect about the first approach is that it didn't make use of the labels, i.e.: the dimensionality reduction was applied solely on information about the text. "Top words are more useful" is nothing more than one of our hypothesis. We have no idea on what words are more useful when doing classification.
- The second approach is better. But one potential problem is that we kept it fixed, meaning that it may not fit in to our problem very well. One solution is to re-train the word embedding together with the weights of the network (simply set `trainable=True`). The training may take a long time thus a GPU is highly recommended.

Here, we decided to use another more general way to reduce dimensionality. We will be using `SelectKBest` from `sklearn` and using `f_classif` to help up pick up k best features (word). 

In [11]:
from sklearn.feature_selection import SelectKBest

In [12]:
DIM = 20000 # Dimensions to keep, a hyper parameter

# Create a feature selector
# By default, f_classif algorithm is used
# Other available options include mutual_info_classif, chi2, f_regression etc. 

selector = SelectKBest(k=20000)

In [13]:
# The feature selector also requires information from labels
# Fit on training data
selector.fit(train_x, train_y)

SelectKBest(k=20000, score_func=<function f_classif at 0x7fe048f92560>)

In [14]:
# Apply to both training data and testing data
train_x = selector.transform(train_x)
test_x = selector.transform(test_x)

In [15]:
# Sanity check
assert train_x.shape == (25000, 20000)
assert test_x.shape == (25000, 20000)

## Build a MLP Model

We use a Multi-Layer Perceptron model, aka Feed Forward Network, which is quite ubiquitous in applications and robust. Deeper networks are usually more powerful, but they are usually more data hungry. Since we don't have much data, a MLP model may work better in this case.

In [16]:
from tensorflow.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, Dropout

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

In [17]:
# This is quite similar to `build_nn_model` in previous notebook,
# except that we add an extra dropout layer after each dense layer

def build_mlp_model(input_dim, layers, output_dim, dropout_rate=0.2):
    # Input layer
    X = Input(shape=(input_dim,))
    
    # Hidden layer(s)
    H = X
    for layer in layers:
        H = Dense(layer, activation='relu')(H)
        H = Dropout(rate=dropout_rate)(H)
    
    # Output layer
    activation_func = 'softmax' if output_dim > 1 else 'sigmoid'
    
    Y = Dense(output_dim, activation=activation_func)(H)
    return Model(inputs=X, outputs=Y)

In [18]:
hyper_params = {
    'learning_rate': 1e-3,  # default for Adam
    'epochs': 1000,
    'batch_size': 64,
    'layers': [64, 32],
    'dim': DIM,
    'dropout_rate': 0.5,
}

In [19]:
mlp_model = build_mlp_model(
    input_dim=hyper_params['dim'],
    layers=hyper_params['layers'],
    output_dim=1,
    dropout_rate=hyper_params['dropout_rate'],
)

mlp_model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20000)]           0         
_________________________________________________________________
dense (Dense)                (None, 64)                1280064   
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total par

## Compile the Model

In [20]:
from tensorflow.keras.optimizers import Adam

In [21]:
mlp_model.compile(
    optimizer=Adam(lr=hyper_params['learning_rate']),
    loss='binary_crossentropy',
    metrics=['acc'],
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## Callbacks

Callbacks (aka hooks) are functions called every N epochs that help you monitor and log the training process. By default, they will be called every 1 epoch. We will be using two common callbacks here: `EarlyStopping` and `ModelCheckpoint`. The first is used to prevent overfitting and the second is used to keep track of the best models we got so far.

In [22]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [23]:
early_stoppping_hook = EarlyStopping(
    monitor='val_loss',  # what metrics to track
    patience=2,  # maximum number of epochs allowed without imporvement on monitored metrics 
)

CPK_PATH = 'model_cpk.hdf5'    # path to store checkpoint

model_cpk_hook = ModelCheckpoint(
    CPK_PATH,
    monitor='val_loss', 
    save_best_only=True,  # Only keep the best model
)

## Train the Model, Hope for the Best

In [24]:
his = mlp_model.fit(
    train_x, 
    train_y, 
    epochs=10,
    validation_data=[test_x, test_y],
    batch_size=hyper_params['batch_size'],
    callbacks=[early_stoppping_hook, model_cpk_hook],
)

print('Training finished')

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 5s 182us/sample - loss: 0.3557 - acc: 0.8564 - val_loss: 0.2330 - val_acc: 0.9059
Epoch 2/10
25000/25000 [==============================] - 4s 173us/sample - loss: 0.1424 - acc: 0.9513 - val_loss: 0.2492 - val_acc: 0.9012
Epoch 3/10
25000/25000 [==============================] - 4s 173us/sample - loss: 0.0832 - acc: 0.9747 - val_loss: 0.2940 - val_acc: 0.8943
Training finished


## Evaluation

Load the best model and do evaluation:

In [25]:
# Load the model checkpoint
mlp_model.load_weights(CPK_PATH)

# Accuracy on validation 
mlp_model.evaluate(test_x, test_y)

25000/25000 [==============================] - 2s 64us/sample - loss: 0.2330 - acc: 0.9059


[0.23295250891685484, 0.90588]

Results are similar to the following screenshot (benchmark run). The exact number may be different as there is some randomization introduced during batching.

<img src='resources/example_validation.png'>